<a href="https://colab.research.google.com/github/uguazelli/langchain/blob/main/single_prompt_structured_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single Prompt with Structured Output
Use one big prompt + parser if:
* You want speed and simplicity
* You can write a well-structured prompt
* You're doing demo, prototype, or cost-sensitive tasks

# Pip Install

In [ ]:
# Install required packages
!pip install -U langchain langchain-community langchain-openai
!pip install pydantic


# Imports

In [2]:
# Imports'
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI # ✅ use langchain_openai, not community
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import json

In [4]:
# Get API key from Colab secrets
api_key = userdata.get('OPENAI_API_KEY')

In [5]:
# Define LLM (completion model like GPT-3.5-turbo-instruct and level of creativity 0.6 of 1)
llm = OpenAI(model="gpt-3.5-turbo-instruct", api_key=api_key, temperature=0.6)

In [6]:
# STEP 1 — Final output schema
class RestaurantBrand(BaseModel):
    name: str = Field(description="The name of the restaurant")
    slogan: str = Field(description="The marketing slogan")
    concept: str = Field(description="Interior and branding concept")
    logo_description: str = Field(description="Description of the logo idea")
    image_prompt: str = Field(description="Prompt to generate the logo image")

parser = PydanticOutputParser(pydantic_object=RestaurantBrand)

In [7]:
# STEP 2 — Prompt asking model to return JSON matching the schema
format_instructions = parser.get_format_instructions()

final_prompt = PromptTemplate.from_template(
    """
You are a branding assistant. I want to open a {cuisine} restaurant.

Generate:
- A beautiful restaurant name
- A catchy slogan
- A branding and interior design concept
- A short description of an ideal logo (colors, font, style)
- A DALL·E-style image generation prompt for that logo

{format_instructions}
""".strip()
)

In [8]:
# Chain: Prompt ➝ LLM ➝ Parser
chain = final_prompt | llm | parser

In [9]:
# Run it
output = chain.invoke({"cuisine": "Brazil", "format_instructions": format_instructions})

# Display results
print(json.dumps(output.model_dump(), indent=2, ensure_ascii=False))

{
  "name": "Sabor do Brasil",
  "slogan": "Experience the vibrant flavors of Brazil",
  "concept": "A modern and lively atmosphere that celebrates the rich culture and cuisine of Brazil. Bold colors, natural textures, and lively music create an immersive dining experience.",
  "logo_description": "The logo is a stylized version of the Brazilian flag, with a vibrant green background and a yellow circle in the center. The name of the restaurant is written in bold, modern font in the center of the circle.",
  "image_prompt": "Generate an image of a stylized Brazilian flag with the name 'Sabor do Brasil' written in bold, modern font in the center."
}
